In [17]:
import networkx as nx
from typing import List
import math
import random
import time
import sympy
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

# Really interesting puzzle and plenty of info online
# mostly as it was used as an NRICH puzzle
# https://nrich.maths.org/factorsandmultiples
# A few resources below 
#
# https://theorie.ikp.physik.tu-darmstadt.de/qcd/numchain.pdf
# https://math.dartmouth.edu/~carlp/divisorgraph.pdf
# https://arxiv.org/pdf/1807.07783.pdf
# http://www.m-hikari.com/imf-2010/37-40-2010/aladdasiIMF37-40-2010.pdf
    

In [2]:
url='https://www.janestreet.com/puzzles/chain-reaction-index/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([(i,j) for i,j in enumerate(y)])
display(Markdown("### "+y[8]+"\n\n"+str("\n".join(y[10:13]))))

### July 2014 : Puzzle

Write down a chain of integers between 1 and 100, with no repetition, such that
if x and y are consecutive numbers in the chain, then x evenly divides y or y
evenly divides x. Here is an example of such a chain, with length 12:
37, 74, 2, 8, 4, 16, 48, 6, 3, 9, 27, 81
What is the longest chain you can find?

In [3]:
primes = list(sympy.primerange(0, 100))

# Question is to find the longest simple path on the divisor graph of the integers up to 100

def longest_simple_path(numbs,a = 0,b = 0 ):
    g = nx.Graph()
    if a == 0:
        a = numbs[0]
    if b == 0:
        b = numbs[-1]
    numbs.sort()
    #print(numbs,a,b)
    
    for i in range(len(numbs)):
        for j in range(i+1,len(numbs)):
            if numbs[j] % numbs[i] == 0:
                g.add_edge(numbs[i],numbs[j])
    
    longest_paths = []
    longest_path_length = 0
    for path in nx.all_simple_paths(g,a,b):
        if len(path) > longest_path_length:
            #print("longest is now",len(path),end=" ")
            longest_path_length = len(path)
            longest_paths.clear()
            longest_paths.append(path)
        if len(path) > 32:
            break
    return longest_paths[0]
           
def bring_in(n,a,b,sequence):
    raw = [i*n for i in range(1,50) if i*n <101 and i*n in sequence]
    series = longest_simple_path(raw,n*a,n*b)
    print("Adding {} uses {} of the {} available. Start: {} End: {}".format(n,len(series),len(raw),series[0],series[-1]))
    return series

def left(x,give_up):
    sequence = [*range(1,101)]
    [sequence.remove(i) for i in give_up]
    [sequence.remove(i) for i in x]
    for i in range(1,len(x)):
        if (x[i] % x[i-1] !=0) & (x[i-1] % x[i] !=0):
            print("FAIL",x[i], x[i-1])
    return sequence
    
print(primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [4]:
# start with the list of integers
sequence = [*range(1,101)]

# give up on primes 37+ 
give_up = [37, 74, 41, 82, 43, 86, 47, 94, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
[sequence.remove(i) for i in give_up]

print("After removing large primes there are {} left".format(len(sequence)))

# for primes above 13 [17,19,23,29,31] use up  4,3,2,1 to capture as many as possible
prime_series =[i*j for i in [1,17,19,23,29,31] for j in [1,2,3,4] if i*j < 101]
prime_ordered = longest_simple_path(prime_series,31*2,17)
[sequence.remove(i) for i in prime_ordered]

print("The are {} in the prime sequence ".format(len(prime_series)))
print("The longest sequence is {} long".format(len(prime_ordered)))

# bring in mutiples of 13 using 5 to link to 17 and use 6 at the end to maximse (choice between linking in with 5 or 7)
link_1 =[17*5,5]
[sequence.remove(i) for i in link_1]
series_13 = bring_in(13,5,6,sequence)
[sequence.remove(i) for i in series_13]

#link between 6 and 9
link_2=[6, 36, 18, 54, 27, 81, 9]
[sequence.remove(i) for i in link_2]

# bring in mutiples of 7 using 9/10
series_7 = bring_in(7,9,10,sequence)
[sequence.remove(i) for i in series_7]

# bring in mutiples of 11 by using 8 at the end
series_11 = bring_in(11,8,7,sequence)  # can have choose either 5 or 7 at the end
[sequence.remove(i) for i in series_11]

# find the longest path between 10 and 8 in the rest
rest= longest_simple_path(sequence,10,8)
[sequence.remove(i) for i in rest]
last_few = []
[sequence.remove(i) for i in last_few]

print("\nFinal Sequence")
print("~~~~~~~~~~~~~~")
final_sequence = prime_ordered+link_1+series_13+link_2+series_7+rest+series_11
print(final_sequence)
print("Length is {}".format(len(set(final_sequence))))

print(left(final_sequence,give_up))

print("\nELEMENTS")
print(prime_ordered)
print(link_1)
print(series_13)
print(link_2)
print(series_7)
print(rest)
print(series_11)


After removing large primes there are 82 left
The are 22 in the prime sequence 
The longest sequence is 21 long
Adding 13 uses 5 of the 7 available. Start: 65 End: 78
Adding 7 uses 12 of the 14 available. Start: 63 End: 70
Adding 11 uses 8 of the 9 available. Start: 88 End: 77

Final Sequence
~~~~~~~~~~~~~~
[62, 31, 93, 1, 57, 19, 38, 76, 2, 58, 29, 87, 3, 69, 23, 46, 92, 4, 68, 34, 17, 85, 5, 65, 13, 52, 26, 78, 6, 36, 18, 54, 27, 81, 9, 63, 21, 42, 84, 28, 56, 14, 98, 49, 7, 35, 70, 10, 20, 100, 50, 25, 75, 15, 45, 90, 30, 60, 12, 72, 24, 48, 96, 32, 64, 16, 80, 40, 8, 88, 44, 22, 66, 33, 99, 11, 77]
Length is 77
[39, 51, 55, 91, 95]

ELEMENTS
[62, 31, 93, 1, 57, 19, 38, 76, 2, 58, 29, 87, 3, 69, 23, 46, 92, 4, 68, 34, 17]
[85, 5]
[65, 13, 52, 26, 78]
[6, 36, 18, 54, 27, 81, 9]
[63, 21, 42, 84, 28, 56, 14, 98, 49, 7, 35, 70]
[10, 20, 100, 50, 25, 75, 15, 45, 90, 30, 60, 12, 72, 24, 48, 96, 32, 64, 16, 80, 40, 8]
[88, 44, 22, 66, 33, 99, 11, 77]


In [8]:
soln1 = [62, 31, 93, 1, 87, 29, 58, 2, 46, 92, 23, 69, 3, 57, 19, 38, 76, 4, 68, 34, 17, 85, 5, 65, 13, 52, 26, 78, 6, 36, 18, 54, 27, 81, 9, 63, 21, 42, 84, 28, 56, 14, 98, 49, 7, 35, 70, 10, 40, 80, 20, 100, 50, 25, 75, 15, 45, 90, 30, 60, 12, 72, 24, 48, 16, 96, 32, 64, 8, 88, 44, 22, 66, 33, 99, 11, 55]
soln2 = [69, 23, 46, 92, 4, 76, 38, 19, 95, 5, 85, 17, 68, 34, 2, 62, 31, 93, 3, 87, 29, 58, 1, 49, 98, 14, 56, 28, 84, 42, 21, 63, 9, 81, 27, 54, 18, 72, 36, 12, 6, 66, 33, 99, 11, 22, 44, 88, 8, 24, 48, 96, 32, 64, 16, 80, 40, 20, 60, 30, 90, 45, 15, 75, 25, 100, 50, 10, 70, 35, 7, 91, 13, 39, 78, 26, 52]
print(left(soln2,give_up))
print(left(soln1,give_up))

[51, 55, 57, 65, 77]
[39, 51, 77, 91, 95]


In [7]:
url='https://www.janestreet.com/puzzles/chain-reaction-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([(i,j) for i,j in enumerate(y)])
display(Markdown("### "+y[8]+"\n\n"+str("\n".join(y[10:11]))))

### July 2014 : Solution

Tough, open-ended puzzle this month! Many came close, but we received 7
submissions with chains of length 77, which is also the longest we could find.
Congratulations to Alexander Chappell, one of the 7, and this month’s randomly
chosen winner of a Jane Street t-shirt! His submission was: 69 23 46 92 4 76 38
19 95 5 85 17 68 34 2 62 31 93 3 87 29 58 1 49 98 14 56 28 84 42 21 63 9 81 27
54 18 72 36 12 6 66 33 99 11 22 44 88 8 24 48 96 32 64 16 80 40 20 60 30 90 45
15 75 25 100 50 10 70 35 7 91 13 39 78 26 52